## Description

This notebook is to investigate how different convergence properties related to each other. The goal is to find the best properties defined that are materials independent and can be used to predict the convergence of a PP. 

The properties that are investigated are:
- For pressure, compare the complex defined SSSP v1 residue volume and the vannila hydrostatic pressure
- For EOS metrics, compare nu wrt AE and nu with ref 200Ry. (Check and assure the guess that delta' and nu are correlated)
- Compare pressure and EOS metrics (nu ref 200Ry)
- Other pair see if those are correlated or not

What I think is, if I tuning the criteria of properties, there will be a cross from A > B to B > A. The different between if A, B are correlated or not is whether their will be a state where A, B are highly linearly correlated. 

The testing data is generated by running full convergence test in the grid of [20:5:200] Ry for all different properties calculation method, then can extract and construct the properties date from the output.
The tested PPs are Hg, Ga, N, Cs, Mn from gbrv, dojo, psl-paw-high and jth, in order to cover PPs from different generated code sources and different type of elements.

This notebook is to compare that selected properties is or not dependent on the structure of the material.
The AiiDA data is stored at group `SI/convergence-properties-compare/<conf>`. conf is BCC/FCC/SC/DC 

In [1]:
from aiida import load_profile
import typing as t

load_profile("2023-08-07")

from aiida import orm

In [86]:
from aiida_sssp_workflow.workflows.convergence.pressure import helper_get_volume_from_pressure_birch_murnaghan
from aiida_sssp_workflow.calculations.calculate_metric import rel_errors_vec_length, _calcDelta
from aiida_sssp_workflow.calculations.calculate_bands_distance import get_bands_distance

paper_scan_list = [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 90, 100, 120, 150, 200]

def extract_data_scan_list1(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergenceDeltaWorkChain':
            lst = []
            for wf2 in wf.called:

                if wf2.process_label == 'helper_delta_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}
    
    for i, wf in enumerate(lst):
        cutoff = scan_list[i]
        if cutoff not in paper_scan_list:
            continue

        i_para = wf.inputs.input_parameters.get_dict()
        r_para = wf.inputs.ref_parameters.get_dict()
    
        # Get the data
        # Ref_200_nu: the nu value w.r.t. the 200 Ry reference
        # Ref_200_deltap: the delta_p value w.r.t. the 200 Ry reference
        res = get_conv_data1(i_para, r_para)
        data[cutoff] = res

    return data, scan_list

def get_conv_data1(i_para, r_para) -> float:
    """Return AE_delta (per atom), AE_nu, REL_nu (the nu compare to the reference not to AE)"""
    ref_V0, ref_B0, ref_B1 = r_para['birch_murnaghan_results']
    V0, B0, B1 = i_para['birch_murnaghan_results']

    AE_nu = i_para['rel_errors_vec_length']
    REFE_nu = r_para['rel_errors_vec_length']
    
    RELA_nu = rel_errors_vec_length(ref_V0, ref_B0, ref_B1, V0, B0, B1)

    return RELA_nu

def extract_data_scan_list2(node):
    real_scan_list = []
    for wf in node.called:
        if wf.process_label == 'ConvergenceBandsWorkChain':
            lst = []
            for wf2 in wf.called:
                if wf2.process_label == 'helper_bands_distence_difference':
                    lst.append(wf2)
                if wf2.process_label == 'convergence_analysis':
                    break
            
            real_scan_list = wf.outputs.output_parameters_wfc_test.get_dict()['ecutwfc']
        
        else:
            # parse_pseudo_info or _CachingConvergenceWorkChain
            continue
        
    expected_scan_list = list(range(20, 201, 5))
    # find what is in expected but not in real
    missing = list(set(expected_scan_list) - set(real_scan_list))
    if missing:
        # raise a warning
        print(f"Warning - the following cutoffs are missing from node {node.pk}: {missing}")
        scan_list = real_scan_list
    else:
        scan_list = expected_scan_list

    data = {}

    for i, wf in enumerate(lst):
        cutoff = scan_list[i]
        if cutoff not in paper_scan_list:
            continue

        band_structure_a = wf.inputs.band_structure_a
        band_parameters_a = wf.inputs.band_parameters_a.get_dict()
        band_structure_b = wf.inputs.band_structure_b
        band_parameters_b = wf.inputs.band_parameters_b.get_dict()
        smearing = wf.inputs.smearing.value
        fermi_shift = wf.inputs.fermi_shift.value
        do_smearing = wf.inputs.do_smearing.value
        spin = wf.inputs.spin.value

        bandsdata_a = {
            "number_of_electrons": band_parameters_a["number_of_electrons"],
            "number_of_bands": band_parameters_a["number_of_bands"],
            "fermi_level": band_parameters_a["fermi_energy"],
            "bands": band_structure_a.get_bands(),
            "kpoints": band_structure_a.get_kpoints(),
            "weights": band_structure_a.get_array("weights"),
        }
        bandsdata_b = {
            "number_of_electrons": band_parameters_b["number_of_electrons"],
            "number_of_bands": band_parameters_b["number_of_bands"],
            "fermi_level": band_parameters_b["fermi_energy"],
            "bands": band_structure_b.get_bands(),
            "kpoints": band_structure_b.get_kpoints(),
            "weights": band_structure_b.get_array("weights"),
        }

        res = get_conv_data2(
            bandsdata_a,
            bandsdata_b,
            smearing,
            fermi_shift,
            do_smearing,
            spin,
        )
        data[cutoff] = res

    return data, scan_list

def get_conv_data2(bandsdata_a, bandsdata_b, smearing, fermi_shift, do_smearing, spin) -> float:
    res = get_bands_distance(
        bandsdata_a,
        bandsdata_b,
        smearing,
        fermi_shift,
        do_smearing,
        spin,
    )

    return res['eta_c']


In [87]:
g = 'SI/convergence-properties-compare/SC'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
SC_all_data1 = {}
SC_all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        data1, scan_list1 = extract_data_scan_list1(node)
        data2, scan_list2 = extract_data_scan_list2(node)
        SC_all_data1[node.pk] = {
            "data": data1,
            "scan_list": scan_list1    
        }
        SC_all_data2[node.pk] = {
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

Warning - the following cutoffs are missing from node 8506679: [160, 105, 130, 75]
Warning - the following cutoffs are missing from node 8506784: [85]
Warning - the following cutoffs are missing from node 8506955: [145, 155, 105]
Warning - the following cutoffs are missing from node 8507056: [20]
Warning - the following cutoffs are missing from node 8507172: [25]
Warning - the following cutoffs are missing from node 8507404: [25, 50, 35]
Warning - the following cutoffs are missing from node 8507447: [35, 20]
Warning - the following cutoffs are missing from node 8507489: [20]
Warning - the following cutoffs are missing from node 8507549: [65]
Warning - the following cutoffs are missing from node 8507641: [25]


In [88]:
g = 'SI/convergence-properties-compare/DC'
gs_nodes = []
gs_nodes.extend(orm.Group.collection.get(label=g).nodes)
    
DC_all_data1 = {}
DC_all_data2 = {}
for node in gs_nodes:
    # give a node and the tuple of criteria
    # return the deducted cutoffs of A and B
    try:
        data1, scan_list1 = extract_data_scan_list1(node)
        data2, scan_list2 = extract_data_scan_list2(node)
        DC_all_data1[node.pk] = {
            "data": data1,
            "scan_list": scan_list1    
        }
        DC_all_data2[node.pk] = {
            "data": data2,
            "scan_list": scan_list2    
        }
    except Exception as e:
        #print(f"Error: {e}")
        #continue
        print(node.pk)
        raise e

Warning - the following cutoffs are missing from node 8292113: [25]
Warning - the following cutoffs are missing from node 8292218: [40, 25, 20, 30]
Warning - the following cutoffs are missing from node 8357978: [25]
Warning - the following cutoffs are missing from node 8358027: [25]
Warning - the following cutoffs are missing from node 8358076: [35, 75, 50, 20, 25, 155]
Warning - the following cutoffs are missing from node 8358097: [25]
Warning - the following cutoffs are missing from node 8358185: [25, 35]


In [89]:
def extract_cutoff(data, scan_list, criteria):
    """Extract the cutoff for pA and pB from a verification workchain

    Args:
        data (dict): the data extracted from the verification workchain
        scan_list (list): the list of cutoffs used in the verification workchain
        criteria (tuple): first element is the criteria for pA, second element is the criteria for pB

    Returns:
        tuple: the cutoff for pA and pB.
    """
    # Get the cutoff of pA and pB
    cut = 200
    for cutoff in reversed(scan_list):
        try: 
            p = data[cutoff]
        except:
            continue
        
        if p > criteria:
            break

        cut = cutoff

    return cut

In [90]:
def compute_cutoff(data12_tuple, criteria):
    cut_A_lst = []
    cut_B_lst = []

    all_data1 = data12_tuple[0]
    all_data2 = data12_tuple[1]
    criteria1 = criteria[0]
    criteria2 = criteria[1]

    for node_pk in all_data1:
        data = all_data1[node_pk]['data']
        scan_list = all_data1[node_pk]['scan_list']
        cut_A = extract_cutoff(data, scan_list, criteria1)
        cut_A_lst.append(cut_A)
    
    for node_pk in all_data2:
        data = all_data2[node_pk]['data']
        scan_list = all_data2[node_pk]['scan_list']
        cut_B = extract_cutoff(data, scan_list, criteria2)
        cut_B_lst.append(cut_B)
        
    return cut_A_lst, cut_B_lst

In [91]:
# Get data for plotting (complex pressure)
SC_cut_A_lst, DC_cut_A_lst = compute_cutoff(data12_tuple=(SC_all_data1, DC_all_data1), criteria=(0.1, 0.1))

In [92]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=SC_cut_A_lst, y=DC_cut_A_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'


In [93]:
conf1_slider = ipw.FloatSlider(value=0.1, min=0.00, max=0.2, step=0.005, description='cA')
conf2_slider = ipw.FloatSlider(value=0.1, min=0.00, max=0.2, step=0.005, description='CB')

def response(change):
    SC_cut_A_lst, DC_cut_A_lst = compute_cutoff(data12_tuple=(SC_all_data1, DC_all_data1), criteria=(conf1_slider.value, conf2_slider.value))
    with g.batch_update():
        g.data[0].x = SC_cut_A_lst
        g.data[0].y = DC_cut_A_lst
        
conf1_slider.observe(response, names="value")
conf2_slider.observe(response, names="value")

slider_widgets = ipw.HBox([conf1_slider, conf2_slider])
app = ipw.VBox([slider_widgets, g])
app

In [94]:
# heatmap
import numpy as np

def compute_correlation(x, y):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data1, DC_all_data1), criteria=(x, y))
    arr_A = np.array(cut_A_lst)
    arr_B = np.array(cut_B_lst)
    
    z = np.sum(np.abs(arr_A - arr_B) / 40.0)
    
    return z

xxs = np.linspace(0.0, 0.5, 201)
yys = np.linspace(0.0, 0.5, 201)
# get z map from xxs and yys
zzs = np.zeros((201, 201))
for i, x in enumerate(xxs):
    for j, y in enumerate(yys):
        zzs[j, i] = compute_correlation(x, y)
        
fig = go.FigureWidget(data=go.Heatmap(z=zzs, x=xxs, y=yys, zmax=10, zmin=0))
fig.layout.xaxis.title = 'cutoff from complex pressure'
fig.layout.yaxis.title = 'cutoff from nu'
fig

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '2c5f000d-593c-4c86-bee9-27ed0fc9b154',
              'x': array([0.    , 0.0025, 0.005 , ..., 0.495 , 0.4975, 0.5   ]),
              'y': array([0.    , 0.0025, 0.005 , ..., 0.495 , 0.4975, 0.5   ]),
              'z': array([[ 0.   , 35.   , 44.625, ..., 78.875, 78.875, 78.875],
                          [31.125, 30.125, 31.25 , ..., 49.25 , 49.25 , 49.25 ],
                          [40.125, 27.625, 25.25 , ..., 40.5  , 40.5  , 40.5  ],
                          ...,
                          [77.125, 42.625, 33.25 , ...,  7.5  ,  7.5  ,  7.5  ],
                          [77.125, 42.625, 33.25 , ...,  7.5  ,  7.5  ,  7.5  ],
                          [77.125, 42.625, 33.25 , ...,  7.5  ,  7.5  ,  7.5  ]]),
              'zmax': 10,
              'zmin': 0}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'cutoff from complex pressure'}},
               'yaxis': {'title': {'text': 'cu

In [95]:
# Get data for plotting (simple pressure)
SC_cut_B_lst, DC_cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data2, DC_all_data2), criteria=(0.1, 0.1))

In [96]:
import ipywidgets as ipw
import plotly.graph_objects as go

trace_corr_scatter = go.Scatter(x=SC_cut_B_lst, y=DC_cut_B_lst, mode='markers', name='cutoff correlation')
trace_xy_line = go.Scatter(x=[0, 200], y=[0, 200], name='x=y')
g = go.FigureWidget(data=[trace_corr_scatter, trace_xy_line])
g.layout.xaxis.title = 'cutoff pA'
g.layout.yaxis.title = 'cutoff pB'

In [97]:
conf1_slider = ipw.FloatSlider(value=0.1, min=0.00, max=20, step=0.5, description='cA')
conf2_slider = ipw.FloatSlider(value=0.1, min=0.00, max=20, step=0.5, description='CB')

def response(change):
    SC_cut_B_lst, DC_cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data2, DC_all_data2), criteria=(conf1_slider.value, conf2_slider.value))
    with g.batch_update():
        g.data[0].x = SC_cut_B_lst
        g.data[0].y = DC_cut_B_lst
        
conf1_slider.observe(response, names="value")
conf2_slider.observe(response, names="value")

slider_widgets = ipw.HBox([conf1_slider, conf2_slider])
app = ipw.VBox([slider_widgets, g])
app

In [98]:
# heatmap
import numpy as np

def compute_correlation(x, y):
    cut_A_lst, cut_B_lst = compute_cutoff(data12_tuple=(SC_all_data2, DC_all_data2), criteria=(x, y))
    arr_A = np.array(cut_A_lst)
    arr_B = np.array(cut_B_lst)
    
    z = np.sum(np.abs(arr_A - arr_B) / 40.0)
    
    return z

xxs = np.linspace(0.0, 20, 201)
yys = np.linspace(0.0, 20, 201)
# get z map from xxs and yys
zzs = np.zeros((201, 201))
for i, x in enumerate(xxs):
    for j, y in enumerate(yys):
        zzs[j, i] = compute_correlation(x, y)
        
fig = go.FigureWidget(data=go.Heatmap(z=zzs, x=xxs, y=yys, zmax=10, zmin=0))
fig.layout.xaxis.title = 'cutoff from complex pressure'
fig.layout.yaxis.title = 'cutoff from nu'
fig

FigureWidget({
    'data': [{'type': 'heatmap',
              'uid': '85b7a69f-1583-4131-bf95-3838bbd91f81',
              'x': array([ 0. ,  0.1,  0.2, ..., 19.8, 19.9, 20. ]),
              'y': array([ 0. ,  0.1,  0.2, ..., 19.8, 19.9, 20. ]),
              'z': array([[ 0.   , 40.25 , 46.   , ..., 79.125, 79.125, 79.125],
                          [36.75 , 32.5  , 35.75 , ..., 42.375, 42.375, 42.375],
                          [42.   , 30.   , 32.   , ..., 37.125, 37.125, 37.125],
                          ...,
                          [78.5  , 38.25 , 32.5  , ...,  4.625,  4.625,  4.625],
                          [78.5  , 38.25 , 32.5  , ...,  4.625,  4.625,  4.625],
                          [78.5  , 38.25 , 32.5  , ...,  4.625,  4.625,  4.625]]),
              'zmax': 10,
              'zmin': 0}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'cutoff from complex pressure'}},
               'yaxis': {'title': {'text': 'cutoff from nu'}}}
})